In [ ]:
!pip install wandb -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.3 MB/s eta 0:00:00


In [ ]:
!pip install wandb --quiet
!pip install python-Levenshtein -q
!git clone --recursive https://github.com/parlance/ctcdecode.git
!pip install wget -q
%cd ctcdecode
!pip install . -q
%cd ..

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 41.2 MB/s eta 0:00:00
Cloning into 'ctcdecode'...
remote: Enumerating objects: 1102, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 1102 (delta 16), reused 32 (delta 14), pack-reused 1063
Receiving objects: 100% (1102/1102), 782.27 KiB | 11.17 MiB/s, done.
Resolving deltas: 100% (529/529), done.
Submodule 'third_party/ThreadPool' (https://github.com/progschj/ThreadPool.git) registered for path 'third_party/ThreadPool'
Submodule 'third_party/kenlm' (https://github.com/kpu/kenlm.git) registered for path 'third_party/kenlm'
Cloning into '/content/ctcdecode/third_party/ThreadPool'...
remote: Enumerating objects: 82, done.        
remote: Total 82 (delta 0), reused 0 (delta 0), pack-reused 82        
Cloning into '/content/ctcdecode/third_party/kenlm'...
remote: Enumerating objects: 14142

In [ ]:
!pip install torchsummaryX

In [ ]:
import torch
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.utils.data import ConcatDataset
from glob import glob
import torchaudio.transforms as tat
import torchvision 
import torchaudio
import wandb

from sklearn.metrics import accuracy_score
import gc

import zipfile
import pandas as pd
from tqdm import tqdm
import os
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import datetime

# imports for decoding and distance calculation
import ctcdecode
import Levenshtein
from ctcdecode import CTCBeamDecoder
import string

import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

In [ ]:
 #TODO: Use the same Kaggle code from HW1P2
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"adritadas","key":"4d8e5e73b5126973227f2e938ba1249f"}') 
    # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c 11-785-s23-hw3p2

In [ ]:
!unzip -q 11-785-s23-hw3p2.zip
!ls

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# ARPABET PHONEME MAPPING
# DO NOT CHANGE
# This overwrites the phonetics.py file.

CMUdict_ARPAbet = {
    "" : " ",
    "[SIL]": "-", "NG": "G", "F" : "f", "M" : "m", "AE": "@", 
    "R"    : "r", "UW": "u", "N" : "n", "IY": "i", "AW": "W", 
    "V"    : "v", "UH": "U", "OW": "o", "AA": "a", "ER": "R", 
    "HH"   : "h", "Z" : "z", "K" : "k", "CH": "C", "W" : "w", 
    "EY"   : "e", "ZH": "Z", "T" : "t", "EH": "E", "Y" : "y", 
    "AH"   : "A", "B" : "b", "P" : "p", "TH": "T", "DH": "D", 
    "AO"   : "c", "G" : "g", "L" : "l", "JH": "j", "OY": "O", 
    "SH"   : "S", "D" : "d", "AY": "Y", "S" : "s", "IH": "I",
    "[SOS]": "[SOS]", "[EOS]": "[EOS]"
}

CMUdict = list(CMUdict_ARPAbet.keys())
ARPAbet = list(CMUdict_ARPAbet.values())


PHONEMES = CMUdict[:-2]
LABELS = ARPAbet[:-2]
print(len(PHONEMES))

In [ ]:
root = "/content/11-785-s23-hw3p2"
partition = "dev-clean"

In [ ]:
torch.cuda.empty_cache()
gc.collect()

In [ ]:
mfcc_dir = sorted(glob(os.path.join(root, partition, 'mfcc','*.npy')))
transcript_dir = sorted(glob(os.path.join(root,partition,'transcript','*.npy')))

In [ ]:
class AudioDataset(torch.utils.data.Dataset):

    # For this homework, we give you full flexibility to design your data set class.
    # Hint: The data from HW1 is very similar to this HW

    #TODO
    def __init__(self,  root, phonemes = PHONEMES, partition= "train-clean-360" ): 
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''

        # Load the directory and all files in them

        #self.mfcc_dir = mfcc_dir
        #self.transcript_dir = transcript_dir

        self.mfcc_files = sorted(glob(os.path.join(root, partition, "mfcc", "*.npy")))
        self.transcript_files = sorted(glob(os.path.join(root, partition, "transcript", "*.npy")))

        self.PHONEMES = PHONEMES
        self.label_to_int = {label: i for i, label in enumerate(self.PHONEMES)}
        self.features = []
        self.labels = []
        for i, transcript_file in enumerate(self.transcript_files):
            mfcc_file = self.mfcc_files[i]

            assert os.path.splitext(os.path.basename(transcript_file))[0] == os.path.splitext(os.path.basename(mfcc_file))[0]

            mfcc = np.load(mfcc_file)
            transcript = np.load(transcript_file)[1:-1]
            transcript = [self.label_to_int[p] for p in transcript]

            # Normalize the mfcc features
            mfcc -= np.mean(mfcc, axis=0)
            mfcc /= (np.std(mfcc, axis=0) + 1e-8)

            self.features.append(mfcc)
            self.labels.append(transcript)





        
       

        # HOW CAN WE REPRESENT PHONEMES? CAN WE CREATE A MAPPING FOR THEM?
        # HINT: TENSORS CANNOT STORE NON-NUMERICAL VALUES OR STRINGS

        #TODO
        # CREATE AN ARRAY OF ALL FEATUERS AND LABELS
        # WHAT NORMALIZATION TECHNIQUE DID YOU USE IN HW1? CAN WE USE IT HERE?
        '''
        You may decide to do this in __getitem__ if you wish.
        However, doing this here will make the __init__ function take the load of
        loading the data, and shift it away from training.
        '''
       

    def __len__(self):
        
        '''
        TODO: What do we return here?
        '''
        return len(self.features)
        

    def __getitem__(self, ind):
        '''
        TODO: RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS

        If you didn't do the loading and processing of the data in __init__,
        do that here.

        Once done, return a tuple of features and labels.
        '''
        mfcc = torch.tensor(self.features[ind], dtype=torch.float)
        transcript = torch.tensor(self.labels[ind], dtype=torch.long)
       
        return mfcc, transcript
        


    def collate_fn(self,batch):
        '''
        TODO:
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
            look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish. 
            Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lenghts of features, 
            and lengths of labels.
        '''
        mfcc, transcript = zip(*batch)
        mfccs_padded = torch.nn.utils.rnn.pad_sequence(mfcc, batch_first=True)
        transcripts_padded = torch.nn.utils.rnn.pad_sequence(transcript, batch_first=True)
        mfcc_lengths = torch.tensor([len(mfcc) for mfcc in mfcc])
        transcript_lengths = torch.tensor([len(transcript) for transcript in transcript])
        return mfccs_padded, transcripts_padded, mfcc_lengths, transcript_lengths




        # You may apply some transformation, Time and Frequency masking, here in the collate function;
        # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
        #                  -> Would we apply transformation on the validation set as well?
        #                  -> Is the order of axes / dimensions as expected for the transform functions?

        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels

In [ ]:
class AudioDatasetTest(torch.utils.data.Dataset):
    def __init__(self, root, phonemes=PHONEMES, partition="test-clean"):
        """
        Initializes the test dataset.
        Inputs:
            root: str
                Path to the root directory containing data.
            partition: str
                Name of the partition. Default is "test-clean".
        """
        self.mfcc_files = sorted(glob(os.path.join(root, partition, "mfcc", "*.npy")))
        self.length = len(self.mfcc_files)
        self.PHONEMES = PHONEMES
        

        

    def __len__(self):
        """
        Returns the length of the test dataset.
        """
        return self.length

    def __getitem__(self, idx):
        """
        Loads a single MFCC file from disk and returns it as a PyTorch tensor.
        """
        mfcc_path = self.mfcc_files[idx]
        mfcc = np.load(mfcc_path)
        mfcc -= np.mean(mfcc, axis=0)
        mfcc /= (np.std(mfcc, axis=0) + 1e-8)
        mfcc_tensor = torch.tensor(mfcc, dtype=torch.float)
       
        return mfcc_tensor

    def collate_fn(self, batch):
        """
        Pads the sequences in the batch and returns the padded batch along with the lengths.
        """
        
        
        mfcc_tensors = [t for t in batch]
        mfccs_padded = torch.nn.utils.rnn.pad_sequence(mfcc_tensors, batch_first=True)
        mfcc_lengths = torch.tensor([len(mfcc) for mfcc in mfcc_tensors])
        return mfccs_padded,  mfcc_lengths





In [ ]:
BATCH_SIZE = 64 # Increase if your device can handle it

#transforms = torchvision.transforms.Compose([
    #torchaudio.transforms.Spectrogram(),
    #torchaudio.transforms.TimeMasking(time_mask_param = 80),
    #torchaudio.transforms.Normalize(),


# set of tranformations
# You may pass this as a parameter to the dataset class above
# This will help modularize your implementation

root = '/content/11-785-s23-hw3p2' 

In [ ]:
# get me RAMMM!!!! 
import gc 
gc.collect()

In [ ]:
# Create instances of AudioDataset for train-clean-360 and train-clean-100 partitions
train_data = AudioDataset(root, phonemes=PHONEMES, partition="train-clean-360")
#train_data2 = AudioDataset(root, phonemes=PHONEMES, partition="train-clean-100")




In [ ]:

# Create a dataset object using the AudioDataset class for the validation data
val_data = AudioDataset(root, phonemes=PHONEMES, partition="dev-clean")

# Create a dataset object using the AudioTestDataset class for the test data
test_data = AudioDatasetTest(root, phonemes=PHONEMES, partition="test-clean")

# Create DataLoaders for train, validation, and test data
train_loader = torch.utils.data.DataLoader(
    train_data, batch_size=BATCH_SIZE, shuffle=True, collate_fn= train_data.collate_fn
)
val_loader = torch.utils.data.DataLoader(
    val_data, batch_size=BATCH_SIZE, shuffle=False, collate_fn= val_data.collate_fn
)
test_loader = torch.utils.data.DataLoader(
    test_data, batch_size=BATCH_SIZE, shuffle=False, collate_fn = test_data.collate_fn
)

# Print dataset and DataLoader information
print("Batch size: ", BATCH_SIZE)
print(f"Train dataset samples = {len(train_data)}, batches = {len(train_loader)}")
print(f"Val dataset samples = {len(val_data)}, batches = {len(val_loader)}")
print(f"Test dataset samples = {len(test_data)}, batches = {len(test_loader)}")



In [ ]:
# sanity check
for data in train_loader:
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, ly.shape)
    break 

In [ ]:
# get me RAMMM!!!! 
import gc 
gc.collect()

In [ ]:
# Utils for network
torch.cuda.empty_cache()

class PermuteBlock(torch.nn.Module):
    def forward(self, x):
        #print(x.shape)
        
        
        return x.transpose(1, 2)

In [ ]:
class pBLSTM(torch.nn.Module):
    def __init__(self, input_size, hidden_size):
        super(pBLSTM, self).__init__()
        self.blstm = torch.nn.LSTM(input_size=input_size*2, hidden_size=hidden_size, num_layers=2, bidirectional=True,batch_first =True)
    
    def forward(self, x_packed):
        #print("In pblstm")

        x, x_lens = pad_packed_sequence(x_packed, batch_first = True) # Pad the packed sequence to get the padded tensor and the lengths tensor
        #print("Shape of x after pad_packed_sequence:", x.shape)
        #print("Shape of x_lens after pad_packed_sequence:", x_lens.shape)
        
        x, x_lens = self.trunc_reshape(x, x_lens) 
        #print("Shape of x after trunc_reshape:", x.shape)
        #print("Shape of x_lens after trunc_reshape:", x_lens.shape)
        
        x_packed = pack_padded_sequence(x, x_lens, batch_first = True, enforce_sorted=False)
        

        x_packed, _ = self.blstm(x_packed) 

        return x_packed

    def trunc_reshape(self, x, x_lens):
      batch_size, time_steps, features = x.size()

      if time_steps % 2 != 0: 
        x = x[:, :-1, :] # Remove the last timestep
        time_steps -= 1


      x = x.reshape(batch_size, time_steps // 2, features*2) 
      x_lens = x_lens // 2 

      return x, x_lens




In [ ]:
class Encoder(torch.nn.Module):
    '''
    The Encoder takes utterances as inputs and returns latent feature representations
    '''
    def __init__(self, input_size, encoder_hidden_size):
        super(Encoder, self).__init__()

        self.embedding = torch.nn.Sequential(
            # Permute: (batch_size, sequence_length, features) -> (batch_size, features, sequence_length)
            PermuteBlock(),
            # 1D CNN layer
            nn.Conv1d(input_size, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(256),
            
            PermuteBlock()
        ) 
  
        self.pBLSTMs = torch.nn.Sequential(
            pBLSTM(256, encoder_hidden_size),
            #LockedDropout(),
            pBLSTM(encoder_hidden_size * 2, encoder_hidden_size),
            #LockedDropout(),
            
            
        ) 

        #self.LSTMs = torch.nn.LSTM(256, encoder_hidden_size,num_layers =1, batch_first =True,bidirectional = True)
            
        
         
    def forward(self, x, x_lens):
        #print("Shape of x in pblstm:Encoder:",x.shape)
        x = self.embedding(x)
        #print("Shape of x after passing through embedding: Encoder:",x.shape)
        #print("Minimum of x_lens and max of x_lens:", min(x_lens), max(x_lens))
        #x = x.permute(0, 2, 1)
        x_packed = pack_padded_sequence(x, x_lens, batch_first= True, enforce_sorted=False)
        x_packed = self.pBLSTMs(x_packed)
      
        #print("Shape of x after passing through pblstm:x_packed:", x_packed.shape)
        encoder_outputs, encoder_lens = pad_packed_sequence(x_packed, batch_first = True)
        #print("Checking for After pBLSTM")
        #print(torch.any(torch.isnan(encoder_outputs)))
        #print(torch.any(torch.isnan(encoder_lens)))

       
        #print(torch.any(torch.isinf(encoder_outputs)))
        #print(torch.any(torch.isinf(encoder_lens)))

        #print("Shape of x after passing through pblstm and pad packed:", encoder_outputs.shape)

        return encoder_outputs, encoder_lens


In [ ]:

torch.cuda.empty_cache()
gc.collect()

In [ ]:
class Decoder(torch.nn.Module):

    def __init__(self, embed_size, output_size=41):
        super().__init__()
        self.mlp = torch.nn.Sequential(
    torch.nn.Linear(embed_size * 2, 2048),
    PermuteBlock(),
    torch.nn.BatchNorm1d(2048),
    PermuteBlock(),
    torch.nn.Dropout(p=0.2),
    torch.nn.GELU(),

    torch.nn.Linear(2048, 1024),
    PermuteBlock(),
    torch.nn.BatchNorm1d(1024),
    PermuteBlock(),
    torch.nn.Dropout(p=0.2),
    torch.nn.GELU(),

    torch.nn.Linear(1024, 512),
    PermuteBlock(),
    torch.nn.BatchNorm1d(512),
    PermuteBlock(),
    torch.nn.Dropout(p=0.2),
    torch.nn.GELU(),

    torch.nn.Linear(512, output_size),
)





        self.softmax = torch.nn.LogSoftmax(dim=2)

    def forward(self, encoder_out):
        # Call your MLP
        #print("Shape of x before MLP:", x.shape)
        mlp_out = self.mlp(encoder_out)
        
        
        #print("Shape of x after MLP:", x.shape)

        # Apply LogSoftmax to the final output of the decoder for the classification
        out = self.softmax(mlp_out)
        
        #print("Shape of x after softmax:",x.shape)

        return out



In [ ]:
import torchaudio

class ASRModel(nn.Module):

    def __init__(self, input_size, embed_size=192, output_size=len(PHONEMES)):
        super().__init__()

        self.augmentations = torch.nn.Sequential(
            PermuteBlock(),
        
            torchaudio.transforms.TimeMasking(time_mask_param=200, iid_masks=True),

            #PermuteBlock(),
         
            torchaudio.transforms.FrequencyMasking(freq_mask_param= 5 , iid_masks=True),

            PermuteBlock()
)
        
        self.encoder = Encoder(input_size, embed_size*2)
        self.decoder = Decoder(embed_size*2, output_size)

    def forward(self, x, lengths_x):
        if self.training:
            x = self.augmentations(x)

        encoder_out, encoder_lens = self.encoder(x, lengths_x)
        #print("Checking for Encoder")
        #print(torch.any(torch.isnan(encoder_out)))
        #print(torch.any(torch.isnan(encoder_lens)))
        #print(torch.any(torch.isinf(encoder_out)))
        #print(torch.any(torch.isinf(encoder_lens)))

        decoder_out = self.decoder(encoder_out)
        #print("Checking for Decoder")
        #print(torch.any(torch.isnan(decoder_out)))
        #print(torch.any(torch.isnan(encoder_lens)))
        
        #print(torch.any(torch.isinf(decoder_out)))
        #print(torch.any(torch.isinf(encoder_lens)))



        return decoder_out, encoder_lens


In [ ]:
torch.cuda.empty_cache()
gc.collect()

In [ ]:
from torchsummaryX import summary

model = ASRModel(
    input_size  = 27,
    embed_size  = 128,
    output_size = len(PHONEMES)
).to(device)

print(model)
summary(model, x.to(device), lx)


In [ ]:
config = {
    "beam_width" : 3,
    "lr" : 1e-4,
    "epochs" : 70
    } # Feel free to add more items here

In [ ]:
# Define CTC loss as the criterion. How would the losses be reduced?
# CTC Loss: https://pytorch.org/docs/stable/generated/torch.nn.CTCLoss.html
# Refer to the handout for hints
criterion = nn.CTCLoss(blank=0, reduction='mean')

# What goes in here?
optimizer = torch.optim.AdamW(model.parameters(), lr=config['lr'], weight_decay=1e-5)

# Declare the decoder. Use the CTC Beam Decoder to decode phonemes
# CTC Beam Decoder Doc: https://github.com/parlance/ctcdecode
# Here is a possible way to create the decoder, but please look up documentation to see what options are available.

decoder = ctcdecode.CTCBeamDecoder(LABELS,beam_width=config['beam_width'], blank_id = 0,log_probs_input = True)

# Learning rate scheduler, if you need it
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

# Mixed Precision, if you need it
scaler = torch.cuda.amp.GradScaler()


In [ ]:


def decode_prediction(output, output_lens, decoder, PHONEME_MAP=LABELS):
    # output = torch.permute(output,(1,0,2))
    
    #print("Shape of output:", output.shape)
    
    beam_results,beam_scores, timesteps, out_seq_len = decoder.decode(output, seq_lens=output_lens)
    #print("Beam results:", beam_results[0][0].shape)

    pred_strings = []
    
    for i in range(output_lens.shape[0]):
        prediction = beam_results[i][0][:out_seq_len[i][0]]
        
        pred_string = "".join(PHONEME_MAP[prediction[k]] for k in range(len(prediction)))
       
        pred_strings.append(pred_string)
    
    return pred_strings


def calculate_levenshtein(output, label, output_lens, label_lens, decoder, PHONEME_MAP=LABELS):
    
    dist = 0
    batch_size = label.shape[0]

    pred_strings = decode_prediction(output, output_lens, decoder, PHONEME_MAP)
    
    for i in range(batch_size):
        pred_string = pred_strings[i]
        label_string = "".join(PHONEME_MAP[label[i][k]] for k in range(label_lens[i]))
        dist += Levenshtein.distance(pred_string, label_string)

    dist /= batch_size
    return dist


In [ ]:
# test code to check shapes

model.eval()
for i, data in enumerate(val_loader, 0):
    x, y, lx, ly = data
    x, y = x.to(device), y.to(device)
    
  
    #print(type(lx))
    #with torch.no_grad():
    h,lh= model(x,lx)
    #h, lh = model(x, lx)
    #print(h.type)
    #lh = torch.full(size=(h.shape[0],), fill_value= 1, dtype=torch.long)
    #lh = lh.to(torch.int64)  # Convert lh to an integral dtype
    #print("Shape of input lenght:",lh.shape)
   

    loss = criterion(torch.permute(h, (1, 0, 2)), y, lh, ly)
    #h = torch.permute(h, (1, 0, 2))
    #print(loss)
    #lh = torch.full(size=(h.shape[0],), fill_value=h.shape[0], dtype=torch.long)

    #print(lh.shape)
    

    #loss = criterion(h, ly, lh, ly)
    #print(loss.item())

           
    #lh = torch.full(size=(h.shape[1],), fill_value= 1, dtype=torch.long)
           
    #h = h.unsqueeze(0) # add an extra dimension to h
    
    #print("Before conversion:")
    #print("input_lengths shape:", lh.shape, "dtype:", lh.dtype)
    #print("target_lengths shape:", ly.shape, "dtype:", ly.dtype)

    # Checking the input_lengths and target_lengths are of shape [batch_size]
    lh = lh.squeeze()
    ly = ly.squeeze()

   # Checking the input_lengths and target_lengths are of type torch.long
    lh = lh.long()
    ly = ly.long()

    #print("After conversion:")
    #print("input_lengths shape:", lh.shape, "dtype:", lh.dtype)
    #print("target_lengths shape:", ly.shape, "dtype:", ly.dtype)



     
    #print(decode_prediction(h,lx , decoder, LABELS))
    print(calculate_levenshtein(h, y, lh, ly, decoder, LABELS))
    break


In [ ]:
wandb.login(key="a0b5e594abd47863b3c9a38a1319b7dbecb14246")

In [ ]:
run = wandb.init(
    name = "early-submission1", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # run_id = ### Insert specific run id here if you want to resume a previous run
    # resume = "must" ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw3p2-ablations", ### Project should be created in your wandb account 
    config = config ### Wandb Config for your run
)

In [ ]:
from tqdm import tqdm
import numpy as np

def train_model(model, train_loader, criterion, optimizer):
    
    model.train()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 

    total_loss = 0

    for i, data in enumerate(train_loader):
        optimizer.zero_grad()

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.cuda.amp.autocast(): 
           h,lh = model(x,lx)
           #print("In main train loop")
           #print(torch.any(torch.isnan(h)))
           #print(torch.any(torch.isnan(lh)))
           #print(torch.any(torch.isinf(h)))
           #print(torch.any(torch.isinf(lh)))
           #h, lh = model(x, lx)
           #print(h.shape)
           #lh = torch.full(size=(h.shape[0],), fill_value= 1, dtype=torch.long)
           #lh = lh.to(torch.int64)  # Convert lh to an integral dtype
           #print("Shape of input lenght:",lh.shape)
   

           loss = criterion(torch.permute(h, (1, 0, 2)), y, lh, ly)
           #print(loss)


        total_loss += loss.item()

        batch_bar.set_postfix(
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            lr="{:.06f}".format(float(optimizer.param_groups[0]['lr'])))

        batch_bar.update() # Update tqdm bar

        # Another couple things you need for FP16. 
        #scaler.scale(loss).backward() # This is a replacement for loss.backward()
        #scaler.step(optimizer) # This is a replacement for optimizer.step()
        #scaler.update() # This is something added just for FP16
        
        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        scaler.step(optimizer) # This is a replacement for optimizer.step()
        scaler.update()

        del x, y, lx, ly, h, lh, loss 
        torch.cuda.empty_cache()
        

    batch_bar.close() # You need this to close the tqdm bar
    
    return total_loss / len(train_loader)



def validate_model(model, val_loader, decoder, phoneme_map= LABELS):

    model.eval()
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    total_loss = 0
    vdist = 0

    for i, data in enumerate(val_loader):

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)
        
        
        with torch.inference_mode():
           h, lh = model(x,lx)
           #h, lh = model(x, lx)
           #print(h.shape)
           #lh = torch.full(size=(h.shape[0],), fill_value= 1, dtype=torch.long)
           
           #lh = lh.to(torch.int64)  # Convert lh to an integral dtype
           #print("Shape of input lenght:",lh.shape)
   

           loss = criterion(torch.permute(h, (1, 0, 2)), y, lh, ly)

      

        total_loss += float(loss)
        vdist += calculate_levenshtein(h, y, lh, ly, decoder, phoneme_map)

        batch_bar.set_postfix(loss="{:.04f}".format(float(total_loss / (i + 1))), dist="{:.04f}".format(float(vdist / (i + 1))))

        batch_bar.update()
    
        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()
        
    batch_bar.close()
    total_loss = total_loss/len(val_loader)
    val_dist = vdist/len(val_loader)
    return total_loss, val_dist

In [ ]:
def save_model(model, optimizer, scheduler, metric, epoch, path):
    torch.save(
        {'model_state_dict'         : model.state_dict(),
         'optimizer_state_dict'     : optimizer.state_dict(),
         'scheduler_state_dict'     : scheduler.state_dict(),
         metric[0]                  : metric[1], 
         'epoch'                    : epoch}, 
         path
    )

def load_model(path, model, metric= 'valid_acc', optimizer= None, scheduler= None):

    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])

    if optimizer != None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if scheduler != None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        
    epoch   = checkpoint['epoch']
    metric  = checkpoint[metric]

    return [model, optimizer, scheduler, epoch, metric]

In [ ]:

last_epoch_completed = 45
start = last_epoch_completed
end = config["epochs"]
best_lev_dist = 4.1598
epoch_model_path = '/content/gdrive/MyDrive/checkpoint'
best_model_path = '/content/gdrive/MyDrive/checkpoint'


In [ ]:
for epoch in range(config['epochs']):
  print("\nEpoch {}/{}".format(epoch+1,config['epochs']))
    
  curr_lr = float(optimizer.param_groups[0]['lr'])

  train_loss = train_model(model, train_loader, criterion, optimizer)
  valid_loss, valid_dist = validate_model(model, val_loader, decoder, LABELS)
  scheduler.step(valid_dist)
  print("\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_loss, curr_lr))
  print("\tVal Dist {:.04f}%\t Val Loss {:.04f}".format(valid_dist, valid_loss))
    

  

  wandb.log({
        'train_loss': train_loss,  
        'valid_dist': valid_dist, 
        'valid_loss': valid_loss, 
        'lr': curr_lr,
        
    })
    
  #save_model(model, optimizer, scheduler, ['valid_dist', val_dist], epoch, epoch_model_path)
  #wandb.save(epoch_model_path)
  #print("Saved epoch model")

  #if val_dist <= best_lev_dist:
    #best_lev_dist = val_dist
    #save_model(model, optimizer, scheduler, ['valid_dist', val_dist], epoch, best_model_path)
    #wandb.save(best_model_path)
    #print("Saved best model")

run.finish()

In [ ]:
valid_loss, valid_dist = validate_model(model, val_loader, decoder, LABELS)
print("\tVal Dist {:.04f}%\t Val Loss {:.04f}".format(valid_dist, valid_loss))

In [ ]:
#TODO: Make predictions

# Follow the steps below:
# 1. Create a new object for CTCBeamDecoder with a larger number of beams
# 2. Get prediction string by decoding the results of the beam decoder

TEST_BEAM_WIDTH = 10 # Use a larger beam width to improve the search for the best path

test_decoder = CTCBeamDecoder(
    labels=LABELS,
    beam_width=TEST_BEAM_WIDTH,
    log_probs_input=True,
)
results = []

model.eval()
print("Testing")
for data in tqdm(test_loader):

    x , lx= data
    
    x = x.to(device)
    
    
  



    

    with torch.no_grad():
     
      h, lh = model(x,lx)
      #h = torch.permute(h, (1, 0, 2))
      prediction_string = decode_prediction(h, lh, test_decoder, LABELS)
      results.extend(prediction_string)  
    
      del x, lx, h, lh
      torch.cuda.empty_cache()

In [ ]:
data_dir = "/content/11-785-s23-hw3p2/test-clean/random_submission.csv"
df = pd.read_csv(data_dir)
df.label = results
df.to_csv('submission.csv', index = False)
! kaggle competitions submit -c automatic-speech-recognition-asr-slack-kaggle -f submission.csv -m "Message"